In [10]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [11]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
import LinearAlgebra: ldiv!, dot
using Test
using ForwardDiff
using SpecialFunctions
using BenchmarkTools
using TransportMap
using QuadGK
using Polynomials
using Distributions
using Random
using LoopVectorization
using Optim
using NLsolve
using MLDataUtils
using MLDataPattern
using Test
using SparseArrays
using QRupdate
using AdaptiveTransportMap: vander, transform!, evaluate, ncoeff, optimize, negative_log_likelihood!, derivative

# using Profile
# using ProfileView

### Third derivative of Physicist Hermite functions

In [169]:
Ne = 100
x = randn(Ne);

In [179]:
P = PhyHermite(5)

PhyHermite(5, PhyPolyHermite(5, ImmutablePolynomial(120.0*x - 160.0*x^3 + 32.0*x^5), false), false)

In [180]:
P.m

5

In [181]:
vander(P, 3, x)

100×6 Array{Float64,2}:
  1.35442    -1.08359   -20.3627   -14.656     282.569     702.421
 -1.25947    -2.3537     20.1559     8.64353  -311.617     286.214
 -0.497446   -5.57444     8.85797   74.695    -160.941   -1106.79
 -0.283112    3.40305    -6.56779  -29.0697    221.771    -319.614
  1.10959     2.92754    -9.22816  -65.6437    -28.5529   1047.04
 -1.23575     2.2823     12.8239   -61.9534    -49.9943   1171.11
  0.964434   -4.21529   -16.5208    45.8018    283.881    -468.111
  0.29965     3.43151     6.3579   -30.3315   -222.105    -293.363
 -1.28028    -2.14294    20.3006     4.64679  -309.111     361.423
  0.503309   -5.56401    -8.95994   74.4684    162.732   -1101.64
 -1.37834     0.278207   18.8698   -37.0879   -216.572    1032.08
 -1.28836    -2.05481    20.3476     2.98995  -307.778     392.187
  0.0399737   2.53292    -9.48937   -1.72461   174.97     -732.643
  ⋮                                                          ⋮
 -1.29061    -2.02953    20.3597     2.51635  -

In [182]:
map(xi->D(P, 3)(xi),x)

100-element Array{Float64,1}:
   702.4214002864783
   286.2136031795143
 -1106.7875682265935
  -319.61398645282173
  1047.0355770637034
  1171.1054879471883
  -468.1109843729741
  -293.36256494746726
   361.4230373091964
 -1101.6362093157252
  1032.0821022961534
   392.1872468493803
  -732.6426488945979
     ⋮
   400.9350401706472
  1191.9443666965208
  -533.6305021019674
 -1112.882381518356
 -1216.6369979910112
  1165.1812310634057
   -83.48388847562316
   569.7275705639765
  1145.277374984825
   -18.178706051042013
  1127.2879422004205
   535.4240751065889

In [156]:
map(xi->D(P, 2)(xi),x) + (2*P.m +1 .- x .^2) .* map(xi->D(P, 0)(xi),x)

100-element Array{Float64,1}:
  0.0
 -5.684341886080802e-14
  1.1368683772161603e-13
 -2.842170943040401e-14
  5.684341886080802e-14
 -4.263256414560601e-14
  7.105427357601002e-15
  0.0
 -1.7053025658242404e-13
 -7.105427357601002e-15
 -5.684341886080802e-14
  0.0
 -5.684341886080802e-14
  ⋮
  0.0
 -2.1316282072803006e-14
 -5.684341886080802e-14
  8.526512829121202e-14
  0.0
  2.842170943040401e-14
  5.684341886080802e-14
 -2.842170943040401e-14
  0.0
 -5.684341886080802e-14
 -7.105427357601002e-14
 -2.842170943040401e-14

In [157]:
map(xi->D(P, 3)(xi),x) + -2*x .* map(xi->D(P, 0)(xi),x) + (2*P.m +1 .- x .^2) .*  map(xi->D(P, 1)(xi),x)

100-element Array{Float64,1}:
  0.0
  0.0
  1.1368683772161603e-13
  4.192202140984591e-13
  2.2737367544323206e-13
 -4.547473508864641e-13
  2.2737367544323206e-13
  2.2737367544323206e-13
  0.0
 -2.2737367544323206e-13
  1.1368683772161603e-13
  0.0
 -2.8421709430404007e-13
  ⋮
  1.1368683772161603e-13
 -1.1368683772161603e-13
  1.1368683772161603e-13
  2.2737367544323206e-13
  2.2737367544323206e-13
 -2.8421709430404007e-13
 -1.5276668818842154e-13
 -5.719869022868806e-13
 -2.2737367544323206e-13
  0.0
 -2.2737367544323206e-13
  2.2737367544323206e-13

In [164]:
map(xi->D(P, 1)(xi),x) - ( 2*P.m*map(xi->D(PhyHermite(P.m-1), 0)(xi),x) - x .* map(xi->D(P, 0)(xi),x))

100-element Array{Float64,1}:
  1.4210854715202004e-14
  1.4210854715202004e-14
  7.105427357601002e-15
  4.263256414560601e-14
  0.0
 -1.4210854715202004e-14
  1.4210854715202004e-14
  1.4210854715202004e-14
 -7.105427357601002e-15
 -1.4210854715202004e-14
  2.842170943040401e-14
  7.105427357601002e-15
 -3.552713678800501e-14
  ⋮
  1.4210854715202004e-14
  0.0
  1.4210854715202004e-14
 -1.4210854715202004e-14
 -1.4210854715202004e-14
 -4.263256414560601e-14
 -2.4868995751603507e-14
 -1.4210854715202004e-14
 -1.4210854715202004e-14
 -7.105427357601002e-15
 -4.263256414560601e-14
  2.842170943040401e-14

In [166]:
map(xi->D(P, 3)(xi),x) + -2*x .* map(xi->D(P, 0)(xi),x) + (2*P.m +1 .- x .^2) .* ( 2*P.m*map(xi->D(PhyHermite(P.m-1), 0)(xi),x) - x .* map(xi->D(P, 0)(xi),x))

100-element Array{Float64,1}:
 -1.1368683772161603e-13
 -2.2737367544323206e-13
  0.0
  2.5579538487363607e-13
  2.2737367544323206e-13
 -2.2737367544323206e-13
  0.0
  0.0
  8.526512829121202e-14
  0.0
 -1.1368683772161603e-13
  0.0
  0.0
  ⋮
 -1.1368683772161603e-13
 -1.1368683772161603e-13
 -1.4210854715202004e-14
  2.2737367544323206e-13
  2.2737367544323206e-13
 -5.684341886080802e-14
  1.1368683772161603e-13
 -5.231370892033738e-13
  0.0
  5.684341886080802e-14
  2.2737367544323206e-13
 -1.1368683772161603e-13

In [168]:
map(xi->D(P, 3)(xi),x) - (-2*P.m*(2*P.m +1 .- x .^2) .* map(xi->D(PhyHermite(P.m-1), 0)(xi),x) + (2*P.m +3 .- x .^2) .*x .* map(xi->D(P, 0)(xi),x))

100-element Array{Float64,1}:
 -1.1368683772161603e-13
 -1.1368683772161603e-13
 -5.684341886080802e-14
  2.2737367544323206e-13
  2.2737367544323206e-13
 -2.2737367544323206e-13
  0.0
  0.0
  8.526512829121202e-14
  0.0
 -2.2737367544323206e-13
  0.0
 -1.1368683772161603e-13
  ⋮
 -1.1368683772161603e-13
 -1.1368683772161603e-13
 -5.684341886080802e-14
  2.2737367544323206e-13
  1.1368683772161603e-13
 -5.684341886080802e-14
  8.526512829121202e-14
 -5.115907697472721e-13
  0.0
  5.684341886080802e-14
  4.547473508864641e-13
 -1.1368683772161603e-13

### 1D example

In [85]:
    Nx = 3
    Ne = 500
    ens = EnsembleState(Nx, Ne)
    ens.S .= randn(Nx, Ne)

    B = MultiBasis(CstProHermite(3), Nx)

    idx = [0 0 0]
    truncidx = idx[1:2:end,:]
    coeff =  randn(size(idx,1))

    f = ExpandedFunction(B, idx, coeff)

ExpandedFunction(5, 1, 3, MultiBasis(Basis of 5 functions: Constant -> 3th degree Probabilistic Hermite function
, 3), [0 0 0], [-0.9198501466955125])

In [86]:
dxdxkψ = grad_x_grad_xd(f, ens.S, f.idx)

500×3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 ⋮         
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [87]:
dxdxkψ_t = ForwardDiff.hessian(f, member(ens,1))

3×3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [88]:
for i=1:Ne
    dxdxkψ_t = ForwardDiff.hessian(f, member(ens,i))
    for j=1:Nx
        @test abs(dxdxkψ[i,j] - dxdxkψ_t[j,end])<1e-10
    end
end 

### Test Hessian of the log π(x)

In [30]:
Nx = 2
Ne = 8
m = 10

X = randn(Nx, Ne)

idx = [0 0; 0 1; 1 0; 1 1; 1 2]
coeff =   [0.6285037650645056;
 -0.4744029092496623;
  1.1405280011620331;
 -0.7217760771894809;
  0.11855056306742319];

C = MapComponent(m, Nx, idx, coeff);

In [34]:
dψ_basis = grad_x_basis(C.I.f.f, X)

8×5×2 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  -0.207296    -0.130929     0.00117735
 0.0  0.0  -0.216523    -0.134105     0.0375507
 0.0  0.0   0.00618292   0.00360034   0.00205305
 0.0  0.0  -0.199714    -0.0830991   -0.107373
 0.0  0.0   0.209274     0.0991538   -0.106331
 0.0  0.0  -0.236677    -0.10036     -0.126702
 0.0  0.0  -0.270316    -0.120917     0.142049
 0.0  0.0  -0.0785179   -0.0495593    0.00259814

[:, :, 2] =
 0.0   0.00283979  0.0   0.00155384  0.345581
 0.0   0.0867129   0.0   0.0465448   0.31942
 0.0  -0.166026    0.0  -0.104855    0.307967
 0.0  -0.268816    0.0  -0.149159    0.0381479
 0.0   0.254048    0.0   0.138466    0.109751
 0.0  -0.26767     0.0  -0.064575    0.0207737
 0.0   0.262746    0.0   0.0959916   0.0506544
 0.0   0.0165449   0.0   0.0102847   0.391819

In [41]:
dxlogDJ = zeros(Ne, Nx)

ψoffdψxd = grad_xd(C.I.f.f, X)

8-element Array{Float64,1}:
  0.03850011681624252
 -0.036864339594153354
  0.19095459274793916
  0.23970874743411022
 -0.20745155095311177
  0.17605463803346075
 -0.18792702339659972
  0.03117818085903034

In [50]:
grad_x_grad_xd(C.I.f.f, X, C.I.f.f.idx)

8×2 Array{Float64,2}:
 -0.0150962    0.275081
 -0.00172344   0.272766
  0.00109835   0.191537
 -0.0403773   -0.00821251
 -0.0333779    0.127178
 -0.0481415    0.0110568
  0.0468207    0.0774556
 -0.00492957   0.294571

In [51]:
hess_xd(C.I.f.f, X)

8-element Array{Float64,1}:
  0.27508054698884865
  0.2727658334063561
  0.19153741860252596
 -0.008212514607585927
  0.12717812747627785
  0.01105679171823458
  0.07745559049870646
  0.29457111787580165

In [52]:
dxdxkψx_t = ForwardDiff.hessian(C.I.f.f, X[:,1])

2×2 Array{Float64,2}:
 -0.13341    -0.0150962
 -0.0150962   0.275081

In [ ]:
evaluate()

In [16]:
log_pdf(C, X)

8-element Array{Float64,1}:
 -1.890933371333304
 -1.0774421758804786
 -1.3812358207093065
 -2.756566903253647
 -2.2764943782683207
 -3.268114225886461
 -0.9741216255311081
 -1.442529469931936

In [18]:
Cx = evaluate(C, X)

8-element Array{Float64,1}:
 -1.3182090265735815
  0.5518744832820461
  1.0207836492619973
  1.969508900840876
  1.7095646000932772
  2.2193925767103297
  0.06731649217667446
  1.0856697228586412

In [22]:
getidx(C)

5×2 Array{Int64,2}:
 0  0
 0  1
 1  0
 1  1
 1  2

In [21]:
getcoeff(C)

5-element Array{Float64,1}:
  0.6285037650645056
 -0.4744029092496623
  1.1405280011620331
 -0.7217760771894809
  0.11855056306742319

In [23]:
dψ = grad_x_basis(C.I.f.f, X)

8×5×2 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0   0.107478    0.0190908  -0.043005
 0.0  0.0   0.0865389   0.0541193  -0.0107887
 0.0  0.0   0.138011    0.0855902   0.0231549
 0.0  0.0  -0.270698   -0.113793   -0.145219
 0.0  0.0  -0.259926   -0.128072   -0.127645
 0.0  0.0  -0.225594   -0.0869868  -0.122217
 0.0  0.0  -0.266312   -0.155186    0.0881003
 0.0  0.0   0.261878    0.157944    0.0678763

[:, :, 2] =
 0.0   0.200064   0.0   0.122532   -0.167233
 0.0   0.0623342  0.0   0.0386104   0.379666
 0.0  -0.0838875  0.0  -0.0502452   0.357862
 0.0  -0.268231   0.0  -0.105483    0.0306983
 0.0  -0.24554    0.0  -0.112246    0.113373
 0.0  -0.270878   0.0  -0.142266    0.00262786
 0.0   0.165408   0.0   0.0714071   0.211025
 0.0  -0.129595   0.0  -0.0583847   0.246624

In [24]:
d2ψ = hess_x_basis(C.I.f.f, X)

8×5×2×2 Array{Float64,4}:
[:, :, 1, 1] =
 0.0  0.0  -0.287372   -0.0510446    0.114986
 0.0  0.0  -0.297614   -0.18612      0.0371031
 0.0  0.0  -0.267679   -0.166006    -0.0449099
 0.0  0.0  -0.0102897  -0.00432547  -0.00552002
 0.0  0.0  -0.0807765  -0.0398007   -0.0396678
 0.0  0.0  -0.1657     -0.0638922   -0.089769
 0.0  0.0  -0.0515665  -0.030049     0.0170591
 0.0  0.0  -0.0730321  -0.0440471   -0.0189292

[:, :, 2, 1] =
 0.0   0.0   0.0   0.0215025   -0.0293467
 0.0   0.0   0.0   0.00539434   0.0530439
 0.0  -0.0   0.0  -0.0115774    0.0824581
 0.0  -0.0  -0.0   0.0726096   -0.0211313
 0.0  -0.0  -0.0   0.0638224   -0.0644633
 0.0  -0.0  -0.0   0.0611086   -0.00112877
 0.0   0.0  -0.0  -0.0440502   -0.130179
 0.0  -0.0   0.0  -0.0339381    0.143359

[:, :, 1, 2] =
 0.0   0.0   0.0   0.0215025   -0.0293467
 0.0   0.0   0.0   0.00539434   0.0530439
 0.0  -0.0   0.0  -0.0115774    0.0824581
 0.0  -0.0  -0.0   0.0726096   -0.0211313
 0.0  -0.0  -0.0   0.0638224   -0.0644633
 0.0  -

### Integration

In [20]:
Nx = 100
Ny = 50
m = 20
Ne = 400
X = randn(Nx, Ne).*randn(Nx, Ne) #.+ 0.5*randn(Nx).*cos.(randn(Nx, Ne) .* randn(Nx, Ne))

# L = LinearTransform(X)
# transform!(L, X);
# C = MapComponent(m, Nx)
# @time C, _ = optimize(C, X, 10)

In [24]:
M = HermiteMap(m, X);

In [25]:
@time M_noqr = optimize(M, X, 10; P = serial, start = Ny+1);

  9.550136 seconds (42.61 M allocations: 13.653 GiB, 7.59% gc time)


In [26]:
@time M_qr = optimize(M, X, 10; P = serial, withqr = true, start = Ny+1);

 18.997304 seconds (45.81 M allocations: 21.111 GiB, 4.17% gc time)


In [31]:
function timing()
    M = HermiteMap(m, X)
    @btime begin 
        optimize($M, $X, $10; P = serial, withqr = false, start = Ny+1)
    end
    
    @btime begin 
        optimize($M, $X, $10; P = serial, withqr = true, start = Ny+1)
    end
end

timing (generic function with 1 method)

In [32]:
timing();

  11.257 s (42606958 allocations: 13.65 GiB)
  21.346 s (45813192 allocations: 21.12 GiB)


In [11]:
timing();

  14.597 s (42433511 allocations: 13.37 GiB)


In [29]:
timing();

  973.450 ms (1054425 allocations: 1.43 GiB)


In [14]:
methods(update_component)

# 1 method for generic function "update_component":
[1] update_component(C::MapComponent, X, reduced_margin::Array{Int64,2}, S::Storage) in AdaptiveTransportMap at /media/mat/HDD/AdaptiveTransportMap/src/mapcomponent/greedyfit.jl:249

In [30]:
function timing()
    @btime begin 
        M = HermiteMap($m, $X)
        optimize($M, $X, $10; P = thread)
    end
end

timing (generic function with 1 method)

In [88]:
timing();

  297.730 ms (1000725 allocations: 1.35 GiB)


### Test optimization with QR

In [45]:
Nx = 3
Ne = 8
m = 20

idx = [0 0 0; 0 0 1; 0 1 0; 0 1 1; 0 1 2; 1 0 0]


Nψ = 6
coeff = [ 0.20649582065364197;
         -0.5150990160472986;
          2.630096893080717;
          1.13653076177397;
          0.6725837371023421;
         -1.3126095306624133]
C = MapComponent(m, Nx, idx, coeff; α = 1e-6);

Ne = 100


# The QR decomposition is not unique!

X = randn(Nx, Ne) .* randn(Nx, Ne) + cos.(randn(Nx, Ne)) .* exp.(-randn(Nx, Ne).^2)

L = LinearTransform(X)
transform!(L, X)
S = Storage(C.I.f, X)
F = QRscaling(S)
newidx = [1 1 1]

Snew = update_storage(S, X, newidx)
Fupdated = updateQRscaling(F, Snew)

Fnew = QRscaling(Snew)

@test norm(Fupdated.D - Fnew.D)<1e-8
@test norm(Fupdated.Dinv - Fnew.Dinv)<1e-8

Test Passed